In [17]:
from langchain import hub
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import FAISS
from langchain.embeddings import OpenAIEmbeddings
from langchain.retrievers import BM25Retriever, EnsembleRetriever
from langchain.chat_models import ChatOpenAI
from langchain.chains import create_qa_with_sources_chain
from langchain.prompts import PromptTemplate
from dotenv import load_dotenv
import os
from pytubefix import YouTube
import asyncio
import torch
from faster_whisper import WhisperModel

In [18]:
load_dotenv()  # .env 파일에서 환경 변수 로드

# OpenAI API 키 설정
os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")

In [19]:
def get_video_info(url):
    yt = YouTube(url)
    audio_stream = yt.streams.filter(only_audio=True).first()
    return {
        "title": yt.title,
        "audio_url": audio_stream.url if audio_stream else None
    }


In [20]:
video_url = "https://www.youtube.com/shorts/a--NSC19MXM"
video_info = get_video_info(video_url)
print(f"Video Title: {video_info['title']}")

Video Title: 가장 쉬운 까르보나라


In [21]:
device = "cuda" if torch.cuda.is_available() else "cpu"
compute_type = "float16" if device == "cuda" else "int8"

In [22]:
whisper_model = WhisperModel("large-v3", device=device, compute_type=compute_type)

def transcribe_audio(audio_url):
    segments, info = whisper_model.transcribe(audio_url)
    transcript = [{"text": segment.text, "start": segment.start, "end": segment.end} for segment in segments]
    return {"script": transcript, "language": info.language}

In [23]:
transcript = transcribe_audio(video_info['audio_url'])
print(f"Transcript Language: {transcript['language']}")
print(f"First few lines of transcript: {transcript['script'][:3]}")

transcribe.py       :324  2024-10-12 11:30:14,287 Processing audio with duration 00:59.118
transcribe.py       :425  2024-10-12 11:30:19,512 Detected language 'ko' with probability 1.00


Transcript Language: ko
First few lines of transcript: [{'text': ' 한 남자가 베이컨을 가져오는데요', 'start': 0.0, 'end': 10.46}, {'text': ' 갑자기 계란을 가져와 깨부수기 시작합니다', 'start': 10.46, 'end': 12.74}, {'text': ' 노른자를 건져내 따로 담아줍니다', 'start': 12.74, 'end': 14.74}]


In [24]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=100, chunk_overlap=10)
summary_prompt = hub.pull("teddynote/summary-stuff-documents-korean")
llm = ChatOpenAI(
            model_name="gpt-4o-mini",
            temperature=0.7,
            streaming=True,
        )

/var/folders/c4/0f7nfvt16ln8630csjtkk_1w0000gn/T/ipykernel_41602/2953908480.py:3: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  llm = ChatOpenAI(


In [25]:
from langchain.chains.combine_documents import create_stuff_documents_chain

In [26]:
from langchain_community.document_loaders import JSONLoader, TextLoader
docs = TextLoader("script.txt").load()

In [27]:
docs

[Document(metadata={'source': 'script.txt'}, page_content="In the last chapter, you and I started to step through the internal workings of a transformer.\nThis is one of the key pieces of technology inside large language models, and a lot of\nother tools in the modern wave of AI.\nIt first hit the scene in a now-famous 2017 paper called Attention is All You Need, and\nin this chapter, you and I will dig into what this attention mechanism is, visualizing how\nit processes data.\nAs a quick recap, here's the important context I want you to have in mind.\nThe goal of the model that you and I are studying is to take in a piece of text and predict\nwhat word comes next.\nThe input text is broken up into little pieces that we call tokens, and these are very often\nwords or pieces of words, but just to make the examples in this video easier for you\nand me to think about, let's simplify by pretending that tokens are always just words.\nThe first step in a transformer is to associate each toke

In [28]:
from langchain_core.documents import Document
document = [Document(page_content="\n".join([t["text"] for t in transcript["script"]]))]

In [29]:
summary_chain = create_stuff_documents_chain(llm,summary_prompt)
result = await summary_chain.ainvoke({"context": document})

_client.py          :1786 2024-10-12 11:31:22,301 HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


In [30]:
result

'- 🥓 한 남자가 베이컨을 가져옴.  \n- 🥚 계란을 깨고 노른자를 따로 담음.  \n- 🥄 흰자는 생으로 먹고, 소금을 넣음.  \n- 🌳 나무젓가락을 사용함.  \n- 🍝 파스타를 넣고 재료를 섞음.  \n- 🧂 후추와 그라라빠다노를 뿌림.  \n- 🔥 베이컨을 구워 계란치즈 소스를 섞음.  \n- 🍽️ 까르보나라를 예쁘게 담아 완성함.  \n- 🤤 저도 한번 꼭 먹어보고 싶네요.  '

In [31]:
embeddings = OpenAIEmbeddings()

/var/folders/c4/0f7nfvt16ln8630csjtkk_1w0000gn/T/ipykernel_41602/2497576997.py:1: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import OpenAIEmbeddings``.
  embeddings = OpenAIEmbeddings()


In [32]:
result.strip().split("\n")

['- 🥓 한 남자가 베이컨을 가져옴.  ',
 '- 🥚 계란을 깨고 노른자를 따로 담음.  ',
 '- 🥄 흰자는 생으로 먹고, 소금을 넣음.  ',
 '- 🌳 나무젓가락을 사용함.  ',
 '- 🍝 파스타를 넣고 재료를 섞음.  ',
 '- 🧂 후추와 그라라빠다노를 뿌림.  ',
 '- 🔥 베이컨을 구워 계란치즈 소스를 섞음.  ',
 '- 🍽️ 까르보나라를 예쁘게 담아 완성함.  ',
 '- 🤤 저도 한번 꼭 먹어보고 싶네요.']

In [33]:
documents = text_splitter.create_documents([t["text"] for t in transcript["script"]])
for doc in documents:
    doc.page_content += "\n" + summary.strip()

NameError: name 'summary' is not defined

In [1]:
from dotenv import load_dotenv
import os
load_dotenv()
api_key = os.getenv("RUNPOD_API_KEY")
you_url = "https://youtube.com/shorts/a--NSC19MXM?si=yiun-HK_7wX1sNvL"

In [2]:
import requests
import os
from dotenv import load_dotenv


# RunPod RUNSYNC 엔드포인트 URL
url = "https://api.runpod.ai/v2/uq96boxkzy99ev/runsync"

# FastAPI의 /hello 엔드포인트로 요청하기 위한 데이터 (내부적으로 사용할 파라미터 설정)
body = {"input":{
    "api":{
        "method":"POST",
        "endpoint":"/ping",
    },
    "payload":{},
}}
# 요청 헤더에 API 키 추가
headers = {
    "Authorization": f"Bearer {api_key}",
    "Content-Type": "application/json"
}

# RUNSYNC 요청 보내기
response = requests.post(url, json=body, headers=headers)

# 응답 확인
if response.status_code == 200:
    print("Response:", response.json())
else:
    print(f"Error {response.status_code}: {response.text}")


Response: {'id': 'sync-26147017-ff6e-408d-9622-80c484868c42-e1', 'status': 'IN_QUEUE'}


In [7]:
import requests

# 작업 ID (작업 완료된 job ID)
job_id = response.json()['id']

# RunPod API STATUS 엔드포인트 URL
status_url = f"https://api.runpod.ai/v2/wm1xrz07all039/status/{job_id}"


# 요청 헤더에 API 키 추가
headers = {
    "Authorization": f"Bearer {api_key}"
}

# 작업 상태 및 결과 확인 요청 보내기
response = requests.get(status_url, headers=headers)

# 응답 확인
if response.status_code == 200:
    job_result = response.json()
    if job_result.get("status") == "COMPLETED":
        print("Job Completed! Result:", job_result.get("output"))
    else:
        print(f"Job Status: {job_result.get('status')}")
else:
    print(f"Error {response.status_code}: {response.text}")


Job Status: IN_QUEUE


In [5]:
import requests
import os
from dotenv import load_dotenv

# Load environment variables
load_dotenv()
api_key = os.getenv("RUNPOD_API_KEY")
endpoint_id = os.getenv("RUNPOD_ENDPOINT_ID")

# RunPod RUNSYNC 엔드포인트 URL
url = f"https://api.runpod.ai/v2/{endpoint_id}/runsync"
you_url = "https://youtube.com/shorts/a--NSC19MXM?si=cMQBosX_DHsLH7Pd"

# FastAPI의 /hello 엔드포인트로 요청하기 위한 데이터
payload = {
    "input": {
        "endpoint": "/get_script_summary",
        "method": "GET",
        "headers": {"x-session-id": "1234asdf"},
        "params": {"url": you_url},
    }
}

# 요청 헤더에 API 키 추가
headers = {
    "Authorization": f"Bearer {api_key}",
    "Content-Type": "application/json"
}

# RUNSYNC 요청 보내기
response = requests.post(url, json=payload, headers=headers)

# 응답 확인
if response.status_code == 200:
    print("Response:", response.json())
else:
    print(f"Error {response.status_code}: {response.text}")


Response: {'delayTime': 6080, 'executionTime': 14798, 'id': 'sync-e0789d83-3bb3-45c9-bc3f-41a576727ba9-e1', 'output': {'language': 'ko', 'script': [{'end': 10.46, 'start': 0, 'text': ' 한 남자가 베이컨을 가져오는데요'}, {'end': 12.74, 'start': 10.46, 'text': ' 갑자기 계란을 가져와 깨부수기 시작합니다'}, {'end': 14.74, 'start': 12.74, 'text': ' 노른자를 건져내 따로 담아줍니다'}, {'end': 16.68, 'start': 14.74, 'text': ' 남은 흰자는 생으로 먹어버리는데요'}, {'end': 19, 'start': 16.68, 'text': ' 냄비를 가져오더니 소금을 넣어주는데요'}, {'end': 21.26, 'start': 19, 'text': ' 소금을 어 너무 좀 짜지 않나? 쉽게 넣어줍니다'}, {'end': 23.28, 'start': 21.26, 'text': ' 환경을 생각하지 않는 마소킴 씨는'}, {'end': 24.88, 'start': 23.28, 'text': ' 나무젓가락을 가져오는데요'}, {'end': 26.86, 'start': 24.88, 'text': ' 파스타를 가져와 다소곳하게 넣...'}, {'end': 27.98, 'start': 26.86, 'text': ' 좀 더...'}, {'end': 28.8, 'start': 27.98, 'text': ' 헤헤헤헤'}, {'end': 29.34, 'start': 28.8, 'text': ' 이 새끼'}, {'end': 30.24, 'start': 29.34, 'text': ' 노른자 2개'}, {'end': 31.34, 'start': 30.24, 'text': ' 후추를 털어놓더니'}, {'end': 33.68, 'start': 31.34, 'tex

In [3]:
Received_event = {'delayTime': 6062, 'id': 'sync-debf511b-458b-4b5d-a24c-8bd09e70744b-e1', 'input': {'endpoint': '/get_script_summary', 'headers': {'x-session-id': '1234asdf'}, 'method': 'GET', 'params': {'url': 'https://api.runpod.ai/v2/ja1tium2uey29v/runsync'}}, 'status': 'IN_QUEUE'}

In [4]:
Received_event.get("input")

{'endpoint': '/get_script_summary',
 'headers': {'x-session-id': '1234asdf'},
 'method': 'GET',
 'params': {'url': 'https://api.runpod.ai/v2/ja1tium2uey29v/runsync'}}

In [10]:
response.json().get('output').get("summary_result")

'- 🥓 한 남자가 베이컨을 가져온다.  \n- 🥚 갑자기 계란을 깨서 노른자를 따로 담는다.  \n- 🥄 남은 흰자는 생으로 먹는다.  \n- 🧂 소금을 넣지만 짜지 않을까 걱정한다.  \n- 🌳 환경을 생각하지 않고 나무젓가락을 사용한다.  \n- 🍝 파스타를 조심스럽게 넣는다.  \n- 🌶️ 후추와 그라라빠다노를 넣기 시작한다.  \n- 💧 면수와 함께 잘 섞는다.  \n- 🫒 올리브유를 너무 많이 넣는다.  \n- 🔥 야들야들하게 베이컨을 구운 후 계란치즈 소스를 섞는다.  \n- 🍽️ 면과 소스를 잘 섞어 그릇에 담아 맛있는 까르보나라를 완성한다.  \n- 😋 저도 한번 꼭 먹어보고 싶다.'

In [18]:
import json
RUNPOD_API_URL = f"https://api.runpod.ai/v2/k4x3rsxvzzpwjl/runsync"
headers = {
    "Authorization": f"Bearer {api_key}",
    "Content-Type": "application/json",
}
payload = {
    "input": {
        "endpoint": "rag_stream_chat",
        "method": "POST",
        "headers": {"x-session-id": "6774b417-534b-4b4e-9b4b-e425b1cea2f1"},
        "params": {"prompt": "주게가 뭔가요?"},
    }
}

response = requests.post(
    RUNPOD_API_URL, headers=headers, json=payload, stream=True
)

for chunk in response.iter_content(chunk_size=None):
    if chunk:
        chunk_data = chunk.decode("utf-8").strip()
        if chunk_data.startswith("data: "):
            chunk_content = chunk_data[6:]
            if chunk_content == "[DONE]":
                break
            try:
                content = json.loads(chunk_content)
                print("Stream content:", content)
            except json.JSONDecodeError:
                print("Invalid JSON:", chunk_content)


In [1]:
Received_event = {'delayTime': 6318, 'id': 'sync-fd743056-ab0e-4911-9f16-123e76f86b9c-e1', 'input': {'endpoint': 'get_title_hash', 'method': 'GET', 'params': {'url': 'https://youtube.com/shorts/a--NSC19MXM?si=yiun-HK_7wX1sNvL'}}, 'status': 'IN_QUEUE'}

In [3]:
Received_event.get(("input"))

{'endpoint': 'get_title_hash',
 'method': 'GET',
 'params': {'url': 'https://youtube.com/shorts/a--NSC19MXM?si=yiun-HK_7wX1sNvL'}}

In [2]:
Sending_request={'scope': {'type': 'http', 'method': 'POST', 'path': '/rag_stream_chat', 'headers': [(b'x-session-id', b'd65ab8e2-4103-4c30-a8e4-01e98577691b')]}, '_receive': None, '_send': "<function empty_send at 0x720467a1cca0>", '_stream_consumed': False, '_is_disconnected': False, '_form': None, '_json': {'prompt': '주제가뭐가요'}}

Recieved_request={'scope': {'type': 'http', 'method': 'POST', 'path': '/rag_stream_chat', 'headers': [(b'x-session-id', b'd65ab8e2-4103-4c30-a8e4-01e98577691b')]}, '_receive': None, '_send': "<function empty_send at 0x720467a1cca0>", '_stream_consumed': False, '_is_disconnected': False, '_form': None, '_json': {'prompt': '주제가뭐가요'}}

In [3]:
Sending_request.keys()

dict_keys(['scope', '_receive', '_send', '_stream_consumed', '_is_disconnected', '_form', '_json'])

In [4]:
temp = {"a":"b"}

In [5]:
temp.get("c")